# Chapter 2: Scope and Namespaces

Understanding how Python resolves names is critical for professional development. This notebook covers the LEGB rule, namespace inspection, and scope manipulation with `global` and `nonlocal`.

## Section 1: The LEGB Rule

In [ ]:
# LEGB: Local, Enclosing, Global, Built-in
# Python searches for names in this order

# Built-in scope (always there)
print(type(len))  # len is built-in

# Global scope
global_var = "I'm global"

def outer_function():
    # Enclosing scope (for inner functions)
    enclosing_var = "I'm in enclosing scope"
    
    def inner_function():
        # Local scope
        local_var = "I'm local"
        
        # Can access all four scopes
        print(f"Local: {local_var}")
        print(f"Enclosing: {enclosing_var}")
        print(f"Global: {global_var}")
        print(f"Built-in: {len}")
    
    inner_function()

outer_function()

In [ ]:
# Name shadowing - inner scope hides outer scope
value = "global"

def function1():
    value = "function1"
    print(f"In function1: {value}")

def function2():
    # No local value, uses global
    print(f"In function2: {value}")

function1()
function2()
print(f"Global: {value}")

print("\nInner scope names shadow outer scope names")

## Section 2: Inspecting Namespaces

In [ ]:
# globals() returns a dictionary of global names
x = 10
y = 20

print("Some globals():")
g = globals()
# Filter out notebook internals
for key in ['x', 'y', 'outer_function', 'function1']:
    if key in g:
        print(f"  {key}: {g[key]}")

In [ ]:
# locals() returns a dictionary of local names
def show_locals():
    a = 1
    b = 2
    c = 3
    
    local_dict = locals()
    print(f"Local variables: {local_dict}")
    print(f"Keys: {list(local_dict.keys())}")

show_locals()

In [ ]:
# dir() shows available names in current scope
import math

def inspect_scope():
    local_var = "hello"
    
    # Filter to show relevant names
    d = dir()
    filtered = [name for name in d if not name.startswith('_')]
    print(f"Non-private names in local scope: {filtered[:10]}")

inspect_scope()

# dir() on module shows what's available
math_names = [name for name in dir(math) if not name.startswith('_')]
print(f"\nFirst 10 math names: {math_names[:10]}")

## Section 3: Global Keyword

In [ ]:
# global keyword allows modifying global variables from inside a function
counter = 0

def increment_counter():
    global counter
    counter += 1
    print(f"Counter: {counter}")

print(f"Initial: {counter}")
increment_counter()
increment_counter()
increment_counter()
print(f"Final: {counter}")

In [ ]:
# Without global, assignment creates a local variable
config = {"debug": False}

def wrong_attempt():
    config = {"debug": True}  # Creates local variable
    print(f"Local config: {config}")

def correct_attempt():
    global config
    config = {"debug": True}  # Modifies global
    print(f"Global config: {config}")

print(f"Original: {config}")
wrong_attempt()
print(f"After wrong_attempt: {config}")
correct_attempt()
print(f"After correct_attempt: {config}")

In [ ]:
# Mutable globals don't need global keyword (but global is clearer)
data = []  # Mutable

def add_to_list(item):
    data.append(item)  # Modifying, not reassigning

print(f"Initial: {data}")
add_to_list(1)
add_to_list(2)
add_to_list(3)
print(f"Final: {data}")

# But global is clearer about intent
debug_log = []

def log_message(msg):
    global debug_log  # Makes intent explicit
    debug_log.append(msg)

log_message("Starting")
log_message("Processing")
log_message("Done")
print(f"\nDebug log: {debug_log}")

## Section 4: Nonlocal Keyword

In [ ]:
# nonlocal allows modifying enclosing scope variables from nested functions

def make_accumulator():
    total = 0  # Enclosing scope
    
    def add(value):
        nonlocal total  # Modify enclosing scope's total
        total += value
        return total
    
    return add

accumulator = make_accumulator()
print(f"accumulator(5) = {accumulator(5)}")
print(f"accumulator(3) = {accumulator(3)}")
print(f"accumulator(2) = {accumulator(2)}")

In [ ]:
# Practical example: counter factory
def make_counter(start=0):
    """Create a counter function with state."""
    count = start
    
    def get():
        return count
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    def decrement():
        nonlocal count
        count -= 1
        return count
    
    def reset():
        nonlocal count
        count = start
        return count
    
    return {'get': get, 'increment': increment, 'decrement': decrement, 'reset': reset}

counter1 = make_counter(10)
print(f"Initial: {counter1['get']()}")
print(f"Increment: {counter1['increment']()}")
print(f"Increment: {counter1['increment']()}")
print(f"Decrement: {counter1['decrement']()}")
print(f"Reset: {counter1['reset']()}")

In [ ]:
# Without nonlocal, assignment creates a local variable
def flawed_accumulator():
    total = 0
    
    def add(value):
        total = total + value  # Error: total referenced before assignment!
        return total
    
    return add

try:
    acc = flawed_accumulator()
    acc(5)
except UnboundLocalError as e:
    print(f"Error (as expected): {e}")
    print("\nThis happens because 'total = ...' makes total local,")
    print("but right side tries to read nonexistent local total.")
    print("\nSolution: use 'nonlocal total' at the start of add()")

## Section 5: Function Attributes

In [ ]:
# Functions are objects and can have attributes
def my_function():
    """Example function."""
    pass

# Functions have useful attributes
print(f"__name__: {my_function.__name__}")
print(f"__doc__: {my_function.__doc__}")
print(f"__module__: {my_function.__module__}")
print(f"__code__.co_varnames: {my_function.__code__.co_varnames}")
print(f"__code__.co_argcount: {my_function.__code__.co_argcount}")

In [ ]:
# Add custom attributes to functions
def process_data(data):
    """Process data efficiently."""
    return len(data)

# Store metadata as attributes
process_data.version = "1.0"
process_data.author = "Alice"
process_data.cache_enabled = True

print(f"Function: {process_data.__name__}")
print(f"Version: {process_data.version}")
print(f"Author: {process_data.author}")
print(f"Cache: {process_data.cache_enabled}")

In [ ]:
# Closures capture variables from enclosing scope
def make_function_with_config(config_value):
    """Create a function that remembers config."""
    
    def inner(x):
        return x * config_value
    
    return inner

# Each closure has its own captured variables
times_two = make_function_with_config(2)
times_ten = make_function_with_config(10)

print(f"times_two(5) = {times_two(5)}")
print(f"times_ten(5) = {times_ten(5)}")

# Inspect closure variables
print(f"\ntimes_two.__closure__: {times_two.__closure__}")
if times_two.__closure__:
    print(f"Captured value: {times_two.__closure__[0].cell_contents}")

## Summary

### LEGB Rule
Python searches for names in this order:
1. **Local**: Current function scope
2. **Enclosing**: Enclosing function scope (closures)
3. **Global**: Module level
4. **Built-in**: Python's built-in scope (len, str, int, etc.)

### Key Functions
- `globals()`: Dictionary of global names
- `locals()`: Dictionary of local names
- `dir()`: List of available names

### scope Keywords
- `global`: Modify a global variable from inside a function
- `nonlocal`: Modify an enclosing scope variable from nested function

### Best Practices
1. Minimize global state - prefer function parameters
2. Use `nonlocal` explicitly to signal intent
3. Avoid shadowing names from outer scopes
4. Use closures for stateful functions
5. Understand that mutable global modifications don't need `global` keyword, but it clarifies intent